In [2]:
import numpy as np
import xgboost as xgb
from linxgb import linxgb
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('../data/data_be.csv')
df['date'] = pd.to_datetime(df['date'])
df.index = df['date']
df.drop('date', axis=1, inplace=True)
df

,price,ActualLoad,ForecastLoad,weekday,season,period
date,,,,,,
2020-01-01 00:00:00,38.60,8700.50,8544.00,Wednesday,winter,night
2020-01-01 01:00:00,36.55,8320.25,8185.75,Wednesday,winter,night
2020-01-01 02:00:00,32.32,8000.75,7896.75,Wednesday,winter,night
2020-01-01 03:00:00,30.85,7795.50,7720.50,Wednesday,winter,night
2020-01-01 04:00:00,30.14,7784.00,7660.25,Wednesday,winter,night
...,...,...,...,...,...,...
2022-12-31 19:00:00,-32.27,8171.75,9171.00,Saturday,winter,evening
2022-12-31 20:00:00,6.20,7846.25,8592.75,Saturday,winter,evening
2022-12-31 21:00:00,35.96,7794.00,8693.50,Saturday,winter,evening


In [4]:
for i in range(3):
    df['price_d{}'.format(i+1)] = df['price'].shift((i+1)*24)

df['price_d{}'.format(7)] = df['price'].shift(7*24)

for i in range(3):
    df['ActualLoad_d{}'.format(i+1)] = df['ActualLoad'].shift((i+1)*24)

df['ActualLoad_d{}'.format(7)] = df['ActualLoad'].shift(7*24)

df.dropna(inplace=True)

In [5]:
# use season instead of month
data = df.drop('ActualLoad',axis=1)
data = pd.get_dummies(data, columns = ['weekday', 'season','period'])
data.drop(['weekday_Monday','season_spring','period_morning'],axis=1,inplace=True)
data.columns

data = data.sample(round(0.5*len(df)))

In [6]:
X = data.drop('price',axis=1)
scaler_X = StandardScaler()
scaler_X.fit(X.iloc[:, 0:9])
X_scaled = X.copy()
X_scaled.iloc[:, 0:9] = scaler_X.transform(X.iloc[:, 0:9])
y = data['price']

# scaler_y = StandardScaler()
# scaler_y.fit(y.values.reshape(-1,1))
# y_scaled = scaler_y.transform(y.values.reshape(-1,1))

In [7]:
preds = []
tests = []
y_indices = []
mymae = 0
myrmse = 0
mymase = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


reg = linxgb(n_estimators=2, max_depth=15)
reg.fit(X_train.values,y_train.values)
y_pred = reg.predict(X_test.values)

mse = mean_squared_error( y_test.values , y_pred)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(y_test.values-y_pred))
mae_naive = np.mean(np.abs(y_test-X_test['price_d1']))

y_pred_train= reg.predict(X_train.values)
mae_train = np.mean(np.abs(y_train.values - y_pred_train))
    
    # mymase = mymase + mae/mae_naive
    # mymae = mymae + mae
    # myrmse = rmse + myrmse
    # preds.append(y_pred)
    # tests.append(y_test)
    # y_indices.append(y_test.index)
    
print(f"Test MAE: {mae}")
print(f"Train MAE: {mae}")
print(f'naive mae:{mae_naive}')


    #plot_importance(regressor,importance_type='gain')




Test score: 23.888556899725877
naive mae:17.77671128107075
Test score: 22.146905822397482
naive mae:17.154110898661568
Test score: 28.393105732120816
naive mae:18.577377990430623
Test score: 30.32836762063248
naive mae:18.463665071770333
Test score: 26.094414561776006
naive mae:17.593215311004784
*******************************
ave mae:26.170270127330532
ave rmse:70.67729820115434
ave mase:1.4578086978676659


In [21]:
y_pred = reg.predict(X_test.values)

mse = mean_squared_error( y_test , y_pred)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(y_test-y_pred))
mae_naive = np.mean(np.abs(y_test-X['price_d1'].iloc[test_index]))

In [23]:
mae_naive

18.410030804774742

In [20]:
y_test.values-y_pred

array([ -48.17      ,   -4.7170747 ,    4.54292345, ...,  -13.68304819,
         13.25      , -108.91571429])